# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

#compute cluster
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

#dataset
from azureml.core import Dataset
from sklearn.model_selection import train_test_split

#automl
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

#model deployment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'my-capstone'
experiment=Experiment(ws, experiment_name)

Check the experiment

In [ ]:
experiment

### Computer Cluster

Lets create computer cluster first, if we have any existing cluster we will use that one, otherwise we will create a brand new cluster

In [ ]:
# Choosing a name for the CPU cluster
amlcompute_cluster_name = "my-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 3)

### Create Datasets

IN this project we are going to use **The Heart Failure Dataset** and this datasets can be found in [Kaggle](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data). 

The Heart failure dataset has total 13 columns 
- age
- anaemia
- creatinine_phosphokinase 
- diabetes
- ejection_fraction
- high_blood_pressure
- platelets
- serum_creatinine
- serum_sodium
- sex
- smoking
- time
- DEATH_EVENT.

In [ ]:
ds = Dataset.get_by_name(ws, name='heart-failure')
dataframe = ds.to_pandas_dataframe()
dataframe.describe()

split datasets on 80% for training data and 20% for testing data

In [ ]:
train_data, test_data = train_test_split(dataframe, test_size=0.2, random_state=121)

In [ ]:
train_data.head()

In [ ]:
os.makedirs('data', exist_ok=True)
training_data_path = 'data/trainingdata.csv'
training_dataset.to_csv(training_data_path)

datastore = ws.get_default_datastore()

datastore.upload(src_dir='data', target_path='data')

# create datasets
train_ds = Dataset.Tabular.from_delimited_files(datastore.path(training_data_path))

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# automl settings
automl_settings = {
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
    "enable_early_stopping": True,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1
}

# automl config
automl_config = AutoMLConfig(task = 'classification',
                            experiment_timeout_minutes=30,
                            compute_target = compute_target,
                            training_data = train_ds,
                            label_column_name = 'DEATH_EVENT',
                            featurization= 'auto',
                            path = project_folder,
                            debug_log = "automl_errors.log",
                            **automl_settings)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = remote_run.get_output()
best_run

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service